In [15]:
import pandas as pd

from scipy.stats import iqr

import wrangle_zillow


In [16]:
df = wrangle_zillow.get_zillow_data()

In [17]:
def find_outlier_columns(df):
    """function takes in dataframe and returns list of column names:
    columns that are floats
    columns that are not categorical data values
    """
    outlier_columns = [x for x in df.columns if df[x].dtype == 'float64']
    outlier_columns.remove('longitude')
    outlier_columns.remove('latitude')
    for x in outlier_columns:
        y = len(df[x].value_counts())
        if y < 6:
            outlier_columns.remove(x)
        elif x.find('id') > 0:
            outlier_columns.remove(x)
    return outlier_columns

In [18]:
outlier_columns = find_outlier_columns(df)

In [19]:
def get_thresholds(df=df, columns=outlier_columns, k=3):
    list_upper = []
    list_lower = []
    for col in outlier_columns:
        lower_25 = (df[col].quantile(.25))
        upper_75 = (df[col].quantile(.75))
        IQR = iqr(df[col])*k
        upper_threshold = upper_75 + IQR
        if df[col].min() >= 0:
            lower_threshold = max([lower_25 - IQR, 0])
        else:
            lower_threshold = lower_25 - IQR
        list_upper.append(upper_threshold)
        list_lower.append(lower_threshold)
    return list_upper, list_lower

In [20]:
list_upper, list_lower = get_thresholds(df=df, columns=outlier_columns, k=3)

In [21]:
def replace_outliers(df=df, outliers_columns=outlier_columns, list_upper=list_upper, list_lower=list_lower):
    for i in range(0, len(outlier_columns)):
        col = outlier_columns[i]
        rebuild_col = []
        for x in df[col]:
            if x > list_upper[i]:
                rebuild_col.append(list_upper[i])
            elif x < list_lower[i]:
                rebuild_col.append(list_upper[i])
            elif x > list_lower[i] and x < list_upper[i]:
                rebuild_col.append(x)
            else:
                rebuild_col.append(-50000000000)
        df[col] = rebuild_col
    return df  

In [27]:
df.calculatedfinishedsquarefeet.max()

35640.0

In [26]:
outlier_columns

['architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'heatingorsystemtypeid',
 'lotsizesquarefeet',
 'poolsizesum',
 'pooltypeid2',
 'propertylandusetypeid',
 'rawcensustractandblock',
 'regionidcounty',
 'regionidzip',
 'roomcnt',
 'threequarterbathnbr',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'fireplaceflag',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'landtaxvaluedollarcnt',
 'taxamount',
 'taxdelinquencyyear',
 'censustractandblock',
 'logerror']